In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/emotions-dataset-for-nlp/val.txt
/kaggle/input/emotions-dataset-for-nlp/test.txt
/kaggle/input/emotions-dataset-for-nlp/train.txt


In [3]:
import pandas as pd
import re 
import nltk

df_test = pd.read_csv('/kaggle/input/emotions-dataset-for-nlp/test.txt', header=None, names=['Text'])
df_train=pd.read_csv('/kaggle/input/emotions-dataset-for-nlp/train.txt',header=None,names=['Text'])

In [4]:
df_test.head()

,Text
0,im feeling rather rotten so im not very ambiti...
1,im updating my blog because i feel shitty;sadness
2,i never make her separate from me because i do...
3,i left with my bouquet of red and yellow tulip...
4,i was feeling a little vain when i did this on...


In [5]:
##remove unwanted characters 
df_test['Cleaned_Text'] = df_test['Text'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)
df_train['Cleaned_Text']=df_train['Text'].str.replace(r'^a-zA-Z0-9\s]','',regex=True)

# Remove extra spaces
df_test['Cleaned_Text'] = df_test['Cleaned_Text'].str.replace(r'\s+', ' ', regex=True).str.strip()

df_train['Cleaned_Text'] = df_train['Cleaned_Text'].str.replace(r'\s+', ' ', regex=True).str.strip()



In [6]:
#limit
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 76.5 MB/s eta 0:00:00:00:01:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [7]:
#limit
import spacy
nlp = spacy.load("en_core_web_sm")

def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

df_test['Lemmatized_Text'] = df_test['Text'].apply(lemmatize_text)
df_train['Lemmatized_Text']=df_train['Text'].apply(lemmatize_text)

In [8]:
df_test.head()

,Text,Cleaned_Text,Lemmatized_Text
0,im feeling rather rotten so im not very ambiti...,im feeling rather rotten so im not very ambiti...,I m feel rather rotten so I m not very ambitio...
1,im updating my blog because i feel shitty;sadness,im updating my blog because i feel shittysadness,I m update my blog because I feel shitty;sadness
2,i never make her separate from me because i do...,i never make her separate from me because i do...,I never make she separate from I because I don...
3,i left with my bouquet of red and yellow tulip...,i left with my bouquet of red and yellow tulip...,I leave with my bouquet of red and yellow tuli...
4,i was feeling a little vain when i did this on...,i was feeling a little vain when i did this on...,I be feel a little vain when I do this one;sad...


In [9]:
#genration of target var
def classify_emotion(text):
    if "happy" in text or "joy" in text:
        return "Happy"
    elif "sad" in text or "cry" in text:
        return "Sad"
    else:
        return "Neutral"  # Or any other default category

df_train['Emotion'] = df_train['Lemmatized_Text'].apply(classify_emotion)
y_train = df_train['Emotion']


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
vectorizer = TfidfVectorizer()

# Vectorize the training and test data
X_train_sparse = vectorizer.fit_transform(df_train['Lemmatized_Text'])
X_test_sparse = vectorizer.transform(df_test['Lemmatized_Text'])
print("Shape of X_train_sparse:", X_train_sparse.shape)
print("Shape of X_test_sparse:", X_test_sparse.shape)


Shape of X_train_sparse: (16000, 12877)
Shape of X_test_sparse: (2000, 12877)


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, classification_report

X = df_train['Lemmatized_Text']
y = df_train['Emotion']  

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize the text data
vectorizer = TfidfVectorizer()
X_train_sparse = vectorizer.fit_transform(X_train)
X_test_sparse = vectorizer.transform(X_test)

# Train the model
model = MultinomialNB()
model.fit(X_train_sparse, y_train)

# Make predictions
y_pred = model.predict(X_test_sparse)

# Evaluate the model
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.934375

Classification Report:
              precision    recall  f1-score   support

       Happy       0.90      0.97      0.93      1069
     Neutral       0.93      0.95      0.94      1184
         Sad       0.99      0.87      0.92       947

    accuracy                           0.93      3200
   macro avg       0.94      0.93      0.93      3200
weighted avg       0.94      0.93      0.93      3200



In [12]:
###checking for overfitting and data leakage 
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X_train_sparse, y_train, cv=5)
print("Cross-validation scores:", scores)
print("Average cross-validation score:", scores.mean())


Cross-validation scores: [0.93359375 0.94023437 0.94023437 0.9484375  0.940625  ]
Average cross-validation score: 0.940625


No major signs of overfitting
Model is generalizing well
No data leakage (most likely, assuming proper vectorization and data handling)

In [13]:
from sklearn.metrics import accuracy_score
y_train_pred = model.predict(X_train_sparse)
train_accuracy = accuracy_score(y_train, y_train_pred)

# Predict on test set
y_test_pred = model.predict(X_test_sparse)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Training Accuracy: {train_accuracy:.2f}")
print(f"Test Accuracy: {test_accuracy:.2f}")


Training Accuracy: 0.98
Test Accuracy: 0.93


In [19]:
import pandas as pd

results_df = pd.DataFrame({
    'Lemmatized_Text': X_test,
    'Predicted_Emotion': y_pred
})

results_df.reset_index(drop=True, inplace=True)

results_df.to_csv("emotion_predictions.csv", index=False)
